In [79]:
### Comment Lines and useful info, commands

#grp_inx = list(df[df['make_model'] == 'Audi A3']['Body'].index)
#df[df["make_model"] == "Audi A3"]["body_type"].mode() ### Audi a3 lerdeki en cok body type.
#oto[col_name].iloc[grp_inx] = oto[col_name].iloc[grp_inx].fillna()
##############################################################################################
#[i for i in df.columns if df[i].notna().mean() < 0.5]

##############################################################################################

# in future i will update here to drop columns
# for i in df.columns:
#     if df.index.stop == df[i].isnull().sum():
#         print("------------------------->",i,df[i].isnull().sum())
#     print(i,df[i].isnull().sum())
##############################################################################################
# to see data in desired counts 
#pd.options.display.max_columns = 15
#pd.options.display.max_rows = 6
#############################################################################################
#df[["price"]].sort_values(by="price",axis=0)
#############################################################################################
#Delete rows that price is lower than 500Euros
#df.drop(df[df["price"]<500].index,inplace=True)
#############################################################################################
#df[["price"]].dropna()
#############################################################################################
#sns.boxplot(x='price',data = df)
#############################################################################################
#get_df_wo_outlier(df,"price")
#############################################################################################
#df.columns
#############################################################################################
#fix values at rows by deleting "\n" in it.
#dfdeneme  = df.replace("\n","")
#############################################################################################
#to have a quick look at data, which columns contain what and how many.
#for i in df.columns:
    #print(df[i].value_counts().rename_axis(i +' unique_values').reset_index(name=i +' counts'))
#############################################################################################    
# important_columns = ['make_model',
# 'km',
# 'registration',
# 'Type',
# 'Body',
# 'Gearing Type',
# 'Drive chain',
# 'Fuel',
# 'Consumption',
# 'Comfort & Convenience',
# 'Entertainment & Media',
# 'Extras',
# 'Safety & Security',
# 'Gears',
# 'Country version',
# 'Last Service Date']
#df[important_columns]
#############################################################################################
# if str(df.at[2765, 'Type'] == 'nan'):
#     print("sss")
# else:
#     print("aaa")
#############################################################################################
#get_all_df(df)
#df = df.astype({"Country version" : "str"})
#############################################################################################
#Convert list type columns to new column with list`s elements
# df = list_to_columns_add2(df,"Full Service") #### NO need to convert NAN to List..
# df = list_to_columns_add2(df,"Consumption") #### NO need to convert NAN to List..
# df = list_to_columns_add2(df,"Type") #### NO need to convert NAN to List..
#df  = list_to_columns_add2(df,"Country version")
#############################################################################################
#Nan_to_list(df,"Country version")
#df  = list_to_columns_add2(df,"Country version")
#df[["Country version"]]
#df[["Country version"]] = df[["Country version"]].replace('\\n',np.NaN, regex=True)
#df[["Country version"]]
#############################################################################################
#for i in df.columns:
    #print(i)
#############################################################################################
#print( [i for i in df["Body"][0] if i!='\n'] )
#############################################################################################
#list_of_columns
#non_list_of_columns
#############################################################################################
#pd.options.display.max_columns = 14
#pd.options.display.max_rows = 5
#df.columns.to_list()

# Functions

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import zscore
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
#convert lists values to new columns with this no need to change column type to string or NAN to [] values normally requires before applying below.
def list_to_columns(df,col): 
    dftemp = pd.DataFrame([pd.Series(x) for x in df[col]]) # that is better and quicker. no need to run NAN_to_list.?
    column_names= []
    max_element = len(max(df.loc[df[col].apply(lambda x : isinstance(x, list))][col], key=len))
    #max_element = len(max(df.loc[df[col].notna()][col], key=len))
    for i in range(max_element):
        column_names.append(col + str(i+1))
    dftemp.columns = column_names
    df2 = df.drop(col,axis=1)
    if max_element == 1:
        dftemp.rename(columns={col+'1': col},inplace = True)
    df_added = pd.concat([df2, dftemp], axis=1, sort=False)
    return df_added

In [5]:
# check a column if it contains list of values. returns 2 list that we need to define
def check_is_list(df):
    list_of_columns = []
    non_list_of_columns = []
    for i in df.columns:
        counter_list=0
        counter_non_list=0
        for j in df[i].values:
            if isinstance(j, list):
                counter_list+=1
            else:
                counter_non_list+=1
        if counter_list > 0:
            list_of_columns.append(i)
        else :
            non_list_of_columns.append(i)
    return list_of_columns,non_list_of_columns

In [6]:
#changes a phrase in a list to another value
def del_str_phrase(phrase,liste):
    while(phrase in liste) : 
        liste.remove(phrase)

In [7]:
#get the df colon without outliers specified
def get_df_wo_outlier(df,col,range=1.5):
    Q1 = df[col].quantile(q=0.25)
    Q3 = df[col].quantile(q=0.75)
    IQR = Q3-Q1
    return df[col][~((df[col] <  Q1-(range*IQR)) |(df[col] > Q3+(range*IQR))) ]

In [8]:
#to have a look at data, which columns contain what and how many.
def get_all_df(df):
    for i in df.columns:
        print(df[i].value_counts(dropna = False).rename_axis(i +' unique_values').reset_index(name=i +' counts'))

In [251]:
def get_col_details(df,col):
    print("VALUE COUNTS")
    print(df[col].value_counts(dropna=False))
    print("-----------------------\n","UNIQUE VALUES")
    print(df[col].unique())
    print("-----------------------\n","NULL SUM")
    print(df[col].isnull().sum())
    print("-----------------------\n","NULL Percentage")
    print(df[col].isnull().sum()*100 /df[col].index.stop )
    print("-----------------------\n","COLUMN with NONE-NULL-NAN")
    print(df.query(f'{col} != {col}')[col])

In [9]:
#updates values NAN to empty list : []
def Nan_to_list(df,col):
    Nan_rows = df[col].isnull()
    df.loc[Nan_rows,col] = df.loc[Nan_rows,col].apply(lambda x : [])

In [10]:
def outlier_zscore(df, col, min_z=1, max_z = 5, step = 0.1, print_list = False):
    z_scores = zscore(df[col].dropna())
    threshold_list = []
    for threshold in np.arange(min_z, max_z, step):
        threshold_list.append((threshold, len(np.where(z_scores > threshold)[0])))
        df_outlier = pd.DataFrame(threshold_list, columns = ['threshold', 'outlier_count'])
        df_outlier['pct'] = (df_outlier.outlier_count - df_outlier.outlier_count.shift(-1))/df_outlier.outlier_count*100
    plt.plot(df_outlier.threshold, df_outlier.outlier_count)
    best_treshold = round(df_outlier.iloc[df_outlier.pct.argmax(), 0],2)
    outlier_limit = int(df[col].dropna().mean() + (df[col].dropna().std()) * df_outlier.iloc[df_outlier.pct.argmax(), 0])
    percentile_threshold = stats.percentileofscore(df[col].dropna(), outlier_limit)
    plt.vlines(best_treshold, 0, df_outlier.outlier_count.max(), 
               colors="r", ls = ":"
              )
    plt.annotate("Zscore : {}\nValue : {}\nPercentile : {}".format(best_treshold, outlier_limit, 
                                                                   (np.round(percentile_threshold, 3), 
                                                                    np.round(100-percentile_threshold, 3))), 
                 (best_treshold, df_outlier.outlier_count.max()/2))
    #plt.show()
    if print_list:
        print(df_outlier)
    return (plt, df_outlier, best_treshold, outlier_limit, percentile)

In [11]:
def outlier_inspect(df, col, min_z=1, max_z = 5, step = 0.5, max_hist = None, bins = 50):
    fig = plt.figure(figsize=(20, 6))
    fig.suptitle(col, fontsize=16)
    plt.subplot(1,3,1)
    if max_hist == None:
        sns.distplot(df[col], kde=False, bins = 50)
    else :
        sns.distplot(df[df[col]<=max_hist][col], kde=False, bins = 50)
    plt.subplot(1,3,2)
    sns.boxplot(df[col])
    plt.subplot(1,3,3)
    z_score_inspect = outlier_zscore(df, col, min_z=min_z, max_z = max_z, step = step)
    plt.show()

# Entrance

In [12]:
# read json file with lines method.
df = pd.read_json("scout_car.json", lines=True)

In [13]:
#lets see all column and value_counts
#get_all_df(df)

In [15]:
#fix column name by deleting "\n" in it.
df.rename(columns=lambda x: x.replace("\n",""),inplace=True)

# Convert lists to columns and common clean up for [ ],"" etc

In [16]:
#here we figure out which column has values as a list and which not.
list_columns, nonlist_columns = check_is_list(df)

In [17]:
# now we will convert lists to columns, after removing "description" since its just a comment for car and does not affect the price.
list_columns.remove("description")#no need for this
for i in list_columns:
    df = list_to_columns(df,i)

<ipython-input-3-5c5d1b8c6f1f>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dftemp = pd.DataFrame([pd.Series(x) for x in df[col]]) # that is better and quicker. no need to run NAN_to_list.?


In [18]:
#replace all "", [], \n, values to np.Nan or delete if possible in df
for i in df.columns:
    df[i] = df[i].replace("",np.NaN)
    df[i] = df[i].apply(lambda x: np.nan if (type(x) == list and len(x) == 0) else x) # convert [] to np.NAN
    try:
        df[i] = df[i].apply(lambda x: list(map(lambda x: x.strip(), x)) if type(x)==list else x)
    except:
        continue
    df[i] = df[i].apply(lambda x : np.NaN if x=='\n' else x)
    df[i] = df[i].apply(lambda x: (("".join(x.replace("\n",""))) if (type(x)==str and type(x) != float) else x) if x!=np.nan else x)

#repeat same steps 2nd time for check if there list in list at df if yes apply methods.
list_columns, nonlist_columns = check_is_list(df)
list_columns.remove("description")#no need for this
for i in list_columns:
    df = list_to_columns(df,i)

#replace all "", [], \n, values to np.Nan or delete if possible in df
for i in df.columns:
    df[i] = df[i].replace("",np.NaN)
    df[i] = df[i].apply(lambda x: np.nan if (type(x) == list and len(x) == 0) else x) # convert [] to np.NAN
    try:
        df[i] = df[i].apply(lambda x: list(map(lambda x: x.strip(), x)) if type(x)==list else x)
    except:
        continue
    df[i] = df[i].apply(lambda x : np.NaN if x=='\n' else x)
    df[i] = df[i].apply(lambda x: (("".join(x.replace("\n",""))) if (type(x)==str and type(x) != float) else x) if x!=np.nan else x)

In [117]:
#get_all_df(df)

In [116]:
pd.options.display.max_columns = 300
pd.options.display.max_rows = 6
#df.head()

# Drop Columns

In [59]:
#drop columns that has lower than 30 percent valid value in it.
drop_30Perc = [i for i in df.columns if round(df[i].notna().mean()*100,2) < 30]
df.drop(drop_30Perc,axis=1,inplace=True)

In [451]:
#drop unnecessary columns #subjectif decision

#no need = 'url', 'description','Offer Number'
#same info already exist = "make_model","Body2", "prev_owner"
unnecassary_columns = ['url', 'description', 'Offer Number',"make_model","Body2"\
                       ,"prev_owner","Model Code","Weight"\
                      ,"Full Service","Full Service3"]
for i in unnecassary_columns:
    try:
        df.drop(i,axis=1,inplace=True)
    except:
        pass

In [329]:
df.head()

,short_description,body_type,price,vat,km,registration,hp,Make,Type2,Type4,Previous Owners1,Warranty1,Model2,First Registration2,Body Color2,Paint Type,Upholstery,Nr. of Doors,Nr. of Seats,Gearing Type2,Displacement,Cylinders,Drive chain,Fuel2,CO2 Emission1,Emission Class1,Comfort & Convenience1,Comfort & Convenience2,Comfort & Convenience3,Comfort & Convenience4,Comfort & Convenience5,Comfort & Convenience6,Comfort & Convenience7,Comfort & Convenience8,Comfort & Convenience9,Comfort & Convenience10,Comfort & Convenience11,Comfort & Convenience12,Comfort & Convenience13,Comfort & Convenience14,Comfort & Convenience15,Entertainment & Media1,Entertainment & Media2,Entertainment & Media3,Entertainment & Media4,Entertainment & Media5,Entertainment & Media6,Extras1,Extras2,Extras3,Safety & Security1,Safety & Security2,Safety & Security3,Safety & Security4,Safety & Security5,Safety & Security6,Safety & Security7,Safety & Security8,Safety & Security9,Safety & Security10,Safety & Security11,Safety & Security12,Safety & Security13,Safety & Security14,Gears,Country version,Consumption1,Consumption2,Consumption3
0,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013",01/2016,66,Audi,Used,Diesel (Particulate Filter),2,NaN,A1,2016,Black,Metallic,"Cloth, Black",5,5,Automatic,"1,422",3,front,Diesel (Particulate Filter),99 g CO2/km (comb),6,Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Hill Holder,Leather steering wheel,Light sensor,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Bluetooth,Hands-free equipment,On-board computer,Radio,NaN,NaN,Alloy wheels,Catalytic Converter,Voice Control,ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,Xenon headlights,NaN,NaN,3.8 l/100 km (comb),4.3 l/100 km (city),3.5 l/100 km (country)
1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000",03/2017,141,Audi,Used,Gasoline,NaN,NaN,A1,2017,Red,NaN,"Cloth, Grey",3,4,Automatic,"1,798",4,front,Gasoline,129 g CO2/km (comb),6,Air conditioning,Automatic climate control,Hill Holder,Leather steering wheel,Lumbar support,Parking assist system sensors rear,Power windows,Start-stop system,Tinted windows,NaN,NaN,NaN,NaN,NaN,NaN,Bluetooth,Hands-free equipment,On-board computer,Radio,Sound system,NaN,Alloy wheels,Sport seats,Sport suspension,ABS,Central door lock,Central door lock with remote control,Daytime running lights,Driver-side airbag,Electronic stability control,Head airbag,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,7,NaN,5.6 l/100 km (comb),7.1 l/100 km (city),4.7 l/100 km (country)
2,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450",02/2016,85,Audi,Used,Diesel (Particulate Filter),1,NaN,A1,2016,Black,Metallic,"Cloth, Black",4,4,Automatic,"1,598",NaN,front,Diesel (Particulate Filter),99 g CO2/km (comb),6,Air conditioning,Cruise control,Electrical side mirrors,Hill Holder,Leather steering wheel,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Seat heating,Start-stop system,NaN,NaN,MP3,On-board computer,NaN,NaN,NaN,NaN,Alloy wheels,Voice Control,NaN,ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,NaN,NaN,NaN,NaN,3.8 l/100 km (comb),4.4 l/100 km (city),3.4 l/100 km (country)
3,1.4 TDi Design S tronic,Sedans,14500,None,"73,000",08/2016,66,Audi,Used,Diesel (Particulate Filter),1,NaN,A1,2016,Brown,Metallic,NaN,3,4,Automatic,"1,422",3,NaN,Diesel (Particulate Filter),

# Crop extra info from cells

In [464]:
#Delete "km" from km column
try:
    df["km"] = df["km"].apply(lambda x : x.replace(",","").replace("-","0") if type(x)==str else x)
    df['km'] = df['km'].apply(lambda x : x.split("km")[0].strip())
except:
    pass
#Delete "cc" from Displacement column
try:
    df['Displacement'] = df['Displacement'].apply(lambda x : x if pd.isna(x) else x.split("cc")[0].strip())
    df.rename(columns={'Displacement': 'Displacement (cc)'},inplace=True)
    df["Displacement (cc)"] = df["Displacement (cc)"].apply(lambda x : round(int(x.replace(",",""))/1000,1) if type(x)==str else x)
except:
    pass
#Delete "kW" from hp column
try:
    df['hp'] = df['hp'].apply(lambda x : x if pd.isna(x) else x.split("kW")[0].strip())
    df.rename(columns={'hp': 'hp (kW)'},inplace=True)
except:
    pass
#extract 1 digit for emission class like 6 4 etc.
try:
    df['Emission Class1'] = df['Emission Class1'].str.extract('(\d)').rename(columns={0: 'Emission Class Euro'})
except:
    pass

#extract CO2 Emission digits
try:
    df['CO2 Emission1'] = df['CO2 Emission1'].apply(lambda x : x if pd.isna(x) else x.split(" ")[0])
    df.rename(columns={'CO2 Emission1': 'CO2 Emission (g CO2/km (comb))'},inplace=True)
except:
    pass
# extract consumption columns to digits
try:
    df['Consumption1'] = df['Consumption1'].apply(lambda x : x if pd.isna(x) else x.split(" ")[0].strip())
    df.rename(columns={'Consumption1': 'Consumption l/100 km (comb)'},inplace=True)
except:
    pass
try:
    df['Consumption2'] = df['Consumption2'].apply(lambda x : x if pd.isna(x) else x.split(" ")[0].strip())
    df.rename(columns={'Consumption2': 'Consumption l/100 km (city)'},inplace=True)
except:
    pass
try:
    df['Consumption3'] = df['Consumption3'].apply(lambda x : x if pd.isna(x) else x.split(" ")[0].strip())
    df.rename(columns={'Consumption3': 'Consumption l/100 km (country)'},inplace=True)
except:
    pass

In [463]:
df.head(3)

,short_description,body_type,price,vat,km,registration,hp (kW),Make,Type2,Type4,Previous Owners1,Warranty1,Model2,First Registration2,Body Color2,Paint Type,Upholstery,Nr. of Doors,Nr. of Seats,Gearing Type2,Displacement (cc),Cylinders,Drive chain,Fuel2,CO2 Emission1(g CO2/km (comb)),Emission Class1,Comfort & Convenience1,Comfort & Convenience2,Comfort & Convenience3,Comfort & Convenience4,Comfort & Convenience5,Comfort & Convenience6,Comfort & Convenience7,Comfort & Convenience8,Comfort & Convenience9,Comfort & Convenience10,Comfort & Convenience11,Comfort & Convenience12,Comfort & Convenience13,Comfort & Convenience14,Comfort & Convenience15,Entertainment & Media1,Entertainment & Media2,Entertainment & Media3,Entertainment & Media4,Entertainment & Media5,Entertainment & Media6,Extras1,Extras2,Extras3,Safety & Security1,Safety & Security2,Safety & Security3,Safety & Security4,Safety & Security5,Safety & Security6,Safety & Security7,Safety & Security8,Safety & Security9,Safety & Security10,Safety & Security11,Safety & Security12,Safety & Security13,Safety & Security14,Gears,Country version,Consumption l/100 km (comb),Consumption l/100 km (city),Consumption l/100 km (country)
0,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,56013,01/2016,66,Audi,Used,Diesel (Particulate Filter),2,NaN,A1,2016,Black,Metallic,"Cloth, Black",5,5,Automatic,1.4,3,front,Diesel (Particulate Filter),99,6,Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Hill Holder,Leather steering wheel,Light sensor,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Bluetooth,Hands-free equipment,On-board computer,Radio,NaN,NaN,Alloy wheels,Catalytic Converter,Voice Control,ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,Xenon headlights,NaN,NaN,3.8,4.3,3.5
1,1.8 TFSI sport,Sedans,14500,Price negotiable,80000,03/2017,141,Audi,Used,Gasoline,NaN,NaN,A1,2017,Red,NaN,"Cloth, Grey",3,4,Automatic,1.8,4,front,Gasoline,129,6,Air conditioning,Automatic climate control,Hill Holder,Leather steering wheel,Lumbar support,Parking assist system sensors rear,Power windows,Start-stop system,Tinted windows,NaN,NaN,NaN,NaN,NaN,NaN,Bluetooth,Hands-free equipment,On-board computer,Radio,Sound system,NaN,Alloy wheels,Sport seats,Sport suspension,ABS,Central door lock,Central door lock with remote control,Daytime running lights,Driver-side airbag,Electronic stability control,Head airbag,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,7,NaN,5.6,7.1,4.7
2,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,83450,02/2016,85,Audi,Used,Diesel (Particulate Filter),1,NaN,A1,2016,Black,Metallic,"Cloth, Black",4,4,Automatic,1.6,NaN,front,Diesel (Particulate Filter),99,6,Air conditioning,Cruise control,Electrical side mirrors,Hill Holder,Leather steering wheel,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Seat heating,Start-stop system,NaN,NaN,MP3,On-board computer,NaN,NaN,NaN,NaN,Alloy wheels,Voice Control,NaN,ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,NaN,NaN,NaN,NaN,3.8,4.4,3.4


In [ ]:
df['Displacement'].apply(lambda x : x if pd.isna(x) else x.split("cc")[0].strip())

In [461]:
df['CO2 Emission1'].value_counts(dropna=False)

NaN                       2436
120 g CO2/km (comb)        740
99 g CO2/km (comb)         545
                          ... 
14,457 g CO2/km (comb)       1
331 g CO2/km (comb)          1
193 g CO2/km (comb)          1
Name: CO2 Emission1, Length: 123, dtype: int64

In [ ]:
g CO2/km (comb)

In [458]:
df['CO2 Emission1'].apply(lambda x : x if pd.isna(x) else x.split(" ")[0])
df.rename(columns={'CO2 Emission1': 'CO2 Emission1(g CO2/km (comb))'},inplace=True)


0         99
1        129
2         99
        ... 
15916    139
15917    139
15918    153
Name: CO2 Emission1, Length: 15919, dtype: object

In [460]:
#df['CO2 Emission1'].apply(lambda x :  x.split(" ")[0])

In [353]:
#get_col_details(df,"Consumption1")

In [354]:
#get_col_details(df,"Emission Class1")

# Fill Missing Values and Move cell info to title

In [ ]:
#grp_inx = list(df[df['make_model'] == 'Audi A3']['Body'].index)
#df[df["make_model"] == "Audi A3"]["body_type"].mode() ### Audi a3 lerdeki en cok body type.
#oto[col_name].iloc[grp_inx] = oto[col_name].iloc[grp_inx].fillna()

In [355]:
## First registration column to fill now if km is 0 or below 100 etc.

In [ ]:
def fill_most(df, group_col, col_name):
    for group in list(df[group_col].unique()):
        grp_inx = list(df[df[group_col]==group][col_name].index)
        df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[df[group_col]==group][col_name].mode()[0])

# Feature Engineering (Column addition within)

In [ ]:
##registration ile now arasi icin age olarak yeni bir column ekleyelim

# Convert Column types to Numeric, if Categorical get_dummies()